### Feature Selection Preprocessing Steps
1. import data, basic cleaning, reordering of features, split labels from features
2. impute data just to get a list of feature names for later
    1. fix meal_plan NaN
    2. fix veteran_indicator dtypes
    3. fix hall_name NaN
    4. then impute
3. split numerical and categorical features
4. impute missing data
    1. categorical : most frequent
    2. numerical : mean
5. one hot encode categorical data
6. scale numerical data
7. join numerical and categorical data back into one structure

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings
from sklearn.impute import SimpleImputer

### 1. import, clean, reorder, split labels from features

In [2]:
data = pd.read_csv('student_outcome202220.csv')
data.head()

,transfer_hours_bt,total_sfa_hours_bt,begin_of_semester_gpa,college_bt,major_bt,first_enroll_term,new_SAT,ACT,TSI,hs_percentile,...,points_vs_median,content_completion_rate,mid_term_grade,total_time_spent_in_content,number_of_logins,content_topic_visits,total_time_spent,semester_gpa,probation,discontinued
0,85.0,36.0,4.00,ED,DINS,202110,NaN,NaN,PI,NaN,...,0.978119,0.651572,NaN,11828.250000,1102.000000,492.0,63510.0,4.00,0,0
1,8.0,44.0,3.02,ED,KINE,202030,NaN,15.0,CR,67.0,...,0.636521,0.709124,75.0,13958.666667,741.833333,784.0,100776.0,2.25,0,0
2,0.0,18.0,3.00,LA,CONC,202110,NaN,NaN,NCR,NaN,...,0.838883,0.603404,79.0,33045.000000,817.000000,672.0,98771.0,1.50,1,0
3,18.0,65.0,3.28,FA,MUSC,202010,NaN,20.0,PI,67.0,...,0.661765,0.148493,95.0,1152.625000,1084.000000,9.0,9481.0,3.80,0,0
4,60.0,0.0,NaN,LA,CONC,202220,NaN,NaN,PI,NaN,...,0.951052,0.562601,95.0,17334.833333,285.666667,2076.0,122304.0,4.00,0,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9361 entries, 0 to 9360
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   transfer_hours_bt             9361 non-null   float64
 1   total_sfa_hours_bt            9361 non-null   float64
 2   begin_of_semester_gpa         8877 non-null   float64
 3   college_bt                    9361 non-null   object 
 4   major_bt                      9361 non-null   object 
 5   first_enroll_term             9361 non-null   int64  
 6   new_SAT                       4189 non-null   float64
 7   ACT                           2857 non-null   float64
 8   TSI                           9176 non-null   object 
 9   hs_percentile                 5612 non-null   float64
 10  gender                        9360 non-null   float64
 11  race_ethnicity                9361 non-null   object 
 12  admit_code                    9348 non-null   object 
 13  stu

In [4]:
#split X from labels
Xraw = data.iloc[:, :35] #all features of student_outcome202220
y = data.iloc[:, 35:] #semester_gpa, probation, discontinued value
y_sem_gpa = y.iloc[:,:1]
y_prob = y.iloc[:,1:2]
y_disc = y.iloc[:,2:]

y_prob = np.ravel(y_prob)
y_disc = np.ravel(y_disc)



In [11]:
#semester_gpa has some missing values... drop these rows when using semester_gpa as the label!!
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9361 entries, 0 to 9360
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   semester_gpa  8992 non-null   float64
 1   probation     9361 non-null   int64  
 2   discontinued  9361 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 219.5 KB


### 2. fix meal_plan and veteran_indicator then impute just to get all the feature names for later

In [6]:
#make a new category of mealplan since there is no category for student's without a mealplan
Xraw['meal_plan'] = Xraw['meal_plan'].fillna(value = 'None') 


#make a new category of hall_name since there is no category for 'off campus'
Xraw['hall_name'] = Xraw['hall_name'].fillna(value = 'Off_Campus') 

#veteran_indicator has float and int values for the same code
print(Xraw['veteran_indicator'].unique())
Xraw['veteran_indicator'] = Xraw['veteran_indicator'].fillna(value = 'None')
Xraw['veteran_indicator'] = Xraw['veteran_indicator'].replace({'1.0':'1', '2.0':'2', '4.0':'4', '5.0':'5', '6.0':'6', '8.0':'8', '9.0':'9'})
print(Xraw['veteran_indicator'].unique())

[nan '8.0' '8' '5' 'L' '4.0' '1' '5.0' '4' '6.0' '2.0' '9.0' 'V' '1.0' 'H'
 '6' '9' '2']
['None' '8' '5' 'L' '4' '1' '6' '2' '9' 'V' 'H']


In [7]:
X_reorder = Xraw[Xraw.dtypes.sort_values().index]
X_reorder.head(1)
#all numerical features then all categorical features

,first_enroll_term,total_course_hours_attempted,first_gen_status,transfer_hours_bt,number_of_logins,total_time_spent_in_content,mid_term_grade,content_completion_rate,points_vs_median,points,...,gender,hall_name,meal_plan,TSI,race_ethnicity,admit_code,major_bt,college_bt,student_type_code,veteran_indicator
0,202110,12,0,85.0,1102.0,11828.25,NaN,0.651572,0.978119,0.978119,...,1.0,Off_Campus,None,PI,White,T,DINS,ED,T,None


In [10]:
#once I impute, then the column names go away. 
#so i need to do a one hot encoding before imputing simply to have a list of the column names for feature selection later
X_dummies = pd.get_dummies(X_reorder)
feature_list = list(X_dummies.columns.values)
print(X_dummies.shape)
feature_list #scan over the list, make sure nothing looks weird


file = open('featurelist.txt','w')
for item in feature_list:
    file.write(item+"\n")
file.close()

(9361, 209)


In [12]:
len(feature_list)
#make sure the final set after imputed and one hot encoded has 209 entries!

209

### 3. Split numerical and categorical

In [13]:
#imputers cannot work on specific columns, so I have to split the dataframe into numerical and categorical
X_categorical = X_reorder.iloc[:,-9:]
X_numerical = X_reorder.iloc[:,:26]

### 4. Imputing Missing Data

In [14]:
#fill in missing data before one hot encoding
#need to fill categorical data by most frequent and numerical data by mean

#make 2 imputers
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp_freq = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')



X_numerical = imp_mean.fit_transform(X_numerical)
X_categorical = imp_freq.fit_transform(X_categorical)
print("Size of X_categorical with simple imputer by most frequent: " f"{X_categorical.shape}")
print("Size of X_numerical with simple imputer by mean: " f"{X_numerical.shape}")


Size of X_categorical with simple imputer by most frequent: (9361, 9)
Size of X_numerical with simple imputer by mean: (9361, 26)


### 5. One Hot Encoding on Categorical Data

In [15]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X_cat_ohe = enc.fit_transform(X_categorical).toarray()
X_cat_ohe.shape

(9361, 183)

### 6. Scale numerical data

In [16]:
#don't scale categorical data - leave as one hot encoded
scaler = preprocessing.StandardScaler().fit(X_numerical)
X_num_scaled = scaler.transform(X_numerical)

### 7. Join numerical and categorical data back into 1 structure

In [17]:
X_full = np.concatenate([X_num_scaled, X_cat_ohe], axis = 1)
print("concatenated shape: ", X_full.shape)
print("original shape: ", Xraw.shape)
#yay! we have the same amount of features from the feature_list! 208!!

concatenated shape:  (9361, 209)
original shape:  (9361, 35)


### 8. Drop indices where there is no semester gpa

In [21]:
#drop_list = np.argwhere(np.isnan(y_sem_gpa))
#y_sem_gpa starts at (9361,1)
drop_list = np.where(np.isnan(y_sem_gpa))[0]
print(drop_list.shape) #this matches the number of NaN values! yay!

X_gpa = np.delete(X_full, drop_list, 0)
print(X_gpa.shape) #shape matches! 9361 rows - 369 NaN rows = 8992 

y_gpa = y_sem_gpa[~np.isnan(y_sem_gpa).any(axis = 1)]
print(y_gpa.shape)
y_gpa = np.ravel(y_gpa) #makes the shape go from (8992, 1) to (8992, ) so the models don't spit out warnings


(369,)
(8992, 209)
(8992, 1)


### 9. Create csv files to avoid these steps every time

In [30]:
X_full_df = pd.DataFrame(X_full, columns = feature_list)
X_full_df.head()
X_full_df.to_csv('ProcessedFeatures/allProcessedFeatures.csv')

X_gpa_df = pd.DataFrame(X_gpa, columns = feature_list)
X_gpa_df.to_csv('ProcessedFeatures/gpaProcessedFeatures.csv')

y_gpa_df = pd.DataFrame(y_gpa)
y_gpa_df.to_csv('ProcessedFeatures/y_gpa.csv')
y_disc_df = pd.DataFrame(y_disc)
y_disc_df.to_csv('ProcessedFeatures/y_disc.csv')
y_prob_df = pd.DataFrame(y_prob)
y_prob_df.to_csv('ProcessedFeatures/y_prob.csv')




# FEATURE SELECTION MODELS

In [16]:
from sklearn.feature_selection import SelectFromModel #feature selection
from sklearn.feature_selection import SequentialFeatureSelector #feature selection
from sklearn.neural_network import MLPClassifier #neural network
from sklearn.linear_model import LogisticRegression #disc and prob model
from sklearn.linear_model import SGDRegressor #semester gpa model

from sklearn.model_selection import KFold #cross validation
from sklearn.model_selection import cross_val_score #cross validation
from sklearn.metrics import mean_squared_error #RMSE calculations
from sklearn.metrics import get_scorer_names 

import pickle #save models for later, no retraining

#models
kf = KFold(n_splits = 5)
logreg= LogisticRegression(max_iter = 300, solver = 'sag')
sgd = SGDRegressor(random_state = 64, max_iter = 200)
clf = MLPClassifier(solver = 'sgd', alpha = 1e-5, hidden_layer_sizes = (5,2), random_state = 1)

### Baseline check: run with all features

In [51]:
#find accuracy using all features on logreg or sgd-reg
results = cross_val_score(logreg, X_full, y_disc, cv=kf)
print("Disc Accuracy, all features: " , results.mean()*100.0)

results = cross_val_score(logreg, X_full, y_prob, cv=kf)
print("Prob Accuracy, all features: " , results.mean()*100.0)

results = cross_val_score(sgd, X_gpa, y_gpa, cv=kf, scoring = 'neg_root_mean_squared_error')
print("GPA neg RMSE, all features: " , results.mean())

C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Disc Accuracy, all features:  75.68653857562026


C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Prob Accuracy, all features:  87.3198077949813
GPA neg RMSE, all features:  -0.6265992520894181


###  Reverse/Backwards Feature Selection

Fits the model with all the features, determines a score for each feature, and progressively cuts out the worst features.

Stops when the change in model is below a tolerance

#### with neural network from last week

In [ ]:
sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=10, direction="backward").fit(X_full, y)

print(
    "Features selected by backward sequential selection: "
    f"{feature_names[sfs_backward.get_support()]}"
)


C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C

C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Isabel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

#### with logistic regression and sgd regression

In [ ]:
#give SequentialFeatureSelector an unfitted estimator!!
#nevermind, it never got through just the discontinued after 36 hours... give it fitted then check the features individually


#discontinue
with warnings.catch_warnings(record=True):
    logreg_disc= LogisticRegression(max_iter = 150, solver = 'sag').fit(X_full, y_disc)
    sfs_backward_disc = SequentialFeatureSelector(logreg_disc, n_features_to_select=20, direction="backward").fit(X_full, y_disc)
pickle.dump(sfs_backward_disc, open('sfs_backward_disc.pkl', 'wb'))   
print("done with disc")
    
#probation
with warnings.catch_warnings(record=True):
    logreg_prob= LogisticRegression(max_iter = 150, solver = 'sag').fit(X_full, y_prob)
    sfs_backward_prob = SequentialFeatureSelector(logreg_prob, n_features_to_select=20, direction="backward").fit(X_full, y_prob)
pickle.dump(sfs_backward_prob, open('sfs_backward_prob.pkl', 'wb'))    
print("done with prob")

#sem_gpa
with warnings.catch_warnings(record=True):
    sgd_short = SGDRegressor(random_state = 64, max_iter = 100).fit(X_gpa, y_gpa)
    sfs_backward_gpa = SequentialFeatureSelector(sgd_short, n_features_to_select=20, direction="backward", scoring = 'neg_root_mean_squared_error').fit(X_gpa, y_gpa)
pickle.dump(sfs_backward_gpa, open('sfs_backward_gpa.pkl', 'wb'))    
print("done with all!")


In [ ]:
print(sfs_backward_disc.get_feature_names_out())
print(sfs_backward_prob.get_feature_names_out())
print(sfs_backward_gpa.get_feature_names_out())


In [ ]:

p_sfs_backward_disc = pickle.load(open('sfs_backward_disc.pkl', 'rb'))
p_sfs_backward_prob = pickle.load(open('sfs_backward_prob.pkl', 'rb'))
p_sfs_backward_gpa = pickle.load(open('sfs_backward_gpa.pkl', 'rb'))

print("BEST DISCONTINUE FEATURES")
disc_rsfs_index = [0,1,3,5,18,27,74,77,135,164]
for i in disc_rsfs_index:
    print(feature_list[i])

print("\nBEST PROBATION FEATURES")
prob_rsfs_index = [6,9,10,16,18,19,32,48,61,116]
for i in disc_rsfs_index:
    print(feature_list[i])
        
print("\nBEST SEM_GPA FEATURES")
gpa_rsfs_index = []
for i in gpa_rsfs_index:
    print(feature_list[i])

In [87]:
#train and test logreg model with these features, find accuracy
X_sfs_disc = X_full[:, [0, 1, 3, 5, 18, 27, 74, 77, 135, 164]]
kf = KFold(n_splits = 5)
logreg= LogisticRegression()
results = cross_val_score(logreg, X_sfs_disc, y_disc, cv=kf)
print("Disc Accuracy: " , results.mean()*100.0)    
    
    
X_sfs_prob = X_full[:, [6, 9, 10, 16, 18, 19, 32, 48, 61, 116]]
results = cross_val_score(logreg, X_sfs_prob, y_prob, cv=kf)
print("Prob Accuracy: " , results.mean()*100.0)

Disc Accuracy:  76.32740450212421
Prob Accuracy:  86.50792184027637


In [54]:
## test those features on a neural network and see how the accuracy changes
nn_8 = MLPClassifier(solver = 'sgd', alpha = 1e-5, hidden_layer_sizes = (8,2), max_iter = 400, random_state = 1)
nn_5 = MLPClassifier(solver = 'sgd', alpha = 1e-5, hidden_layer_sizes = (5,2), max_iter = 400, random_state = 1)
nn_3 = MLPClassifier(solver = 'sgd', alpha = 1e-5, hidden_layer_sizes = (3,2), max_iter = 400, random_state = 1)



results = cross_val_score(nn_8, X_sfs_disc, y_disc, cv=kf)
print("Disc (8,2) Accuracy: " , results.mean()*100.0)  

results = cross_val_score(nn_5, X_sfs_disc, y_disc, cv=kf)
print("Disc (5,2) Accuracy: " , results.mean()*100.0)  

results = cross_val_score(nn_3, X_sfs_disc, y_disc, cv=kf)
print("Disc (3,2) Accuracy: " , results.mean()*100.0)  

results = cross_val_score(nn_8, X_sfs_prob, y_prob, cv=kf)
print("Prob (8,2) Accuracy: " , results.mean()*100.0)  

results = cross_val_score(nn_5, X_sfs_prob, y_prob, cv=kf)
print("Prob (5,2) Accuracy: " , results.mean()*100.0)  

results = cross_val_score(nn_3, X_sfs_prob, y_prob, cv=kf)
print("Prob (3,2) Accuracy: " , results.mean()*100.0)  

Disc (8,2) Accuracy:  78.83783157875523
Disc (5,2) Accuracy:  78.12210517429418
Disc (3,2) Accuracy:  75.7931822433958
Prob (8,2) Accuracy:  86.62544320779772
Prob (5,2) Accuracy:  86.46515827709102
Prob (3,2) Accuracy:  86.44380786799367


## Select From Model

creates a logistic regression model using all the features given, then pulls out the best performing features from that one training

In [71]:
#for probation status
with warnings.catch_warnings(record=True):
    logreg = LogisticRegression().fit(X_full,y_prob)
    selector = SelectFromModel(estimator=logreg, max_features = 10).fit(X_full, y_prob)
print("best features to predict probation: ", selector.get_feature_names_out())

#for discontinued status
with warnings.catch_warnings(record=True):
    logreg = LogisticRegression().fit(X_full,y_disc)
    selector = SelectFromModel(estimator=logreg, max_features = 10).fit(X_full, y_disc)
print("best features to predict discontinued: ", selector.get_feature_names_out())

#for semester gpa
with warnings.catch_warnings(record=True):
    sgdreg = SGDRegressor(random_state = 64).fit(X_gpa,y_gpa)
    selector = SelectFromModel(estimator=sgdreg, max_features = 10).fit(X_gpa, y_gpa)
print("best features to predict semester_gpa: ", selector.get_feature_names_out())

best features to predict probation:  ['x6' 'x33' 'x36' 'x40' 'x80' 'x117' 'x122' 'x133' 'x176' 'x195']
best features to predict discontinued:  ['x18' 'x27' 'x33' 'x87' 'x108' 'x117' 'x142' 'x168' 'x183' 'x196']
best features to predict semester_gpa:  ['x6' 'x19' 'x71' 'x117' 'x186' 'x189' 'x193' 'x197' 'x201' 'x206']


In [63]:
#use the list from earlier to see what those x features are
print("BEST PROBATION FEATURES")
gpa_ft_index = [6,33,36,40,80,117,122,133,176,195]
for i in gpa_ft_index:
    print(feature_list[i])
    
#admit_code_FT = UG freshman with <15 transfer hours
#CONC = concurrent/not designated
#DBCC = business comminucation and corp. education
#FCSC = family and consumer science
#RNUR = nursing (post RN)
#student_type_code_R = UG-Returning student

BEST PROBATION FEATURES
mid_term_grade
hall_name_Off Campus
meal_plan_100 Block Plan
meal_plan_45 Block Fac/Staff
admit_code_FT
major_bt_CONC
major_bt_DBCC
major_bt_FCSC
major_bt_RNUR
student_type_code_R


In [64]:
print("BEST DISCONTINUED FEATURES")
disc_ft_index = [18, 27, 33, 87, 108, 117, 142, 168, 183, 196]
for i in disc_ft_index:
    print(feature_list[i])
    
#admit_code_OT = UG-Mature Student(OTAS)
#BIOC = biochemistry
#CONC = concurrent/not designated
#GEOL = geology
#INTB = international business
#PBAD = public administration
#TRAN = transient student
#student_type_code_S = UG-transient

BEST DISCONTINUED FEATURES
total_sfa_hours_bt
hall_name_Hall 14
hall_name_Off Campus
admit_code_OT
major_bt_BIOC
major_bt_CONC
major_bt_GEOL
major_bt_PBAD
major_bt_TRAN
student_type_code_S


In [65]:
print("BEST SEM_GPA FEATURES")
gpa_ft_index = [6, 19, 71, 117, 186, 189, 193, 197, 201, 206]
for i in gpa_ft_index:
    print(feature_list[i])
    
#CONC = concurrent/not determined
#ED = college of education
#LA = college of liberal arts
#student type F = UG-New first time freshman
#student type T = Transfer student
#veteran indicator 5 = chapter 25 dep edu program
#veteran indicator L = legacy

BEST SEM_GPA FEATURES
mid_term_grade
begin_of_semester_gpa
race_ethnicity_Unknown or Not Reported
major_bt_CONC
college_bt_ED
college_bt_LA
student_type_code_F
student_type_code_T
veteran_indicator_5
veteran_indicator_L


In [94]:
#find accuracy of logreg on DISCONTINUE with just the pulled features
X_best_disc = X_full[:, [18,32,86, 107,116,141,151,167,182,195]]

results = cross_val_score(logreg, X_best_disc, y_disc, cv=kf)
print("Disc Accuracy (logreg, 5-cv, SelectFromModel ft): " , results.mean()*100.0) 


#find accuracy of logreg on PROBATION with just the pulled features
X_best_prob = X_full[:, [6,35,39,79,96,116,121,132,175,194]]

logreg= LogisticRegression()
results = cross_val_score(logreg, X_best_prob, y_prob, cv=kf)
print("Prob Accuracy (logreg, 5-cv, SelectFromModel ft): " , results.mean()*100.0)



#find RMSE of sem_gpa with just the pulled features
X_best_gpa = X_gpa[:, [6,19,70,116,185,188,192,196,200,205]]

sgdreg = SGDRegressor(random_state = 64)
results = cross_val_score(sgdreg, X_best_gpa, y_gpa, cv=kf, scoring = 'neg_root_mean_squared_error')
print("GPA neg RMSE (sgd reg, 5-cv, SelectFromModel ft): ", results.mean())

Disc Accuracy (logreg, 5-cv, SelectFromModel ft):  73.38975248812407
Prob Accuracy (logreg, 5-cv, SelectFromModel ft):  85.2901100204891
GPA neg RMSE (sgd reg, 5-cv, SelectFromModel ft):  -0.6495363324748097
